In [1]:
#const value
EMPTY = 0  # no object in acell
WHITE = 1  # white storn in a cell
BLACK = 2  # black storn in a sell
WALL = 3  # recognizer for over the board
ThreeD = 3
X = 0  # X-axis
Y = 1  # Y-axis
Z = 2  # Z-axis
symbols = ['E', '○', '✖︎', 'W']  # provisional graphics
players = ["Empty", "White", "Black","none"]  # player name list
DRAW=2

import numpy as np

class Board:
    # borad control and board instance in this class.

    def setCell(self, pos1, col):  # call and set cell player stone, no wall or empty
        if col == WHITE or col == BLACK:
            self.board[pos1[X]][pos1[Y]][pos1[Z]] = col

    def __init__(self, length):  # class values initializer
        self.x = length
        self.y = length
        self.z = length
        self.L = length
        self.board = [[[0] * length for i in range(length)] for i in range(length)]
        self.initialize(length)

    def get_flattend_board(self):#リスト形式でboard情報を得る
        return np.array(self.board).flatten()

    # unflattenなインデックスをflattenにして返す.
    def get_flatten_point(self, flatten_point):
        a = flatten_point[0] * self.L*self.L
        b = a + flatten_point[1] * self.L
        ret = b + flatten_point[2]
        return ret

    # flattenなインデックスをunflatternなポイントに返す
    def get_unflatten_point(self, i):
        z = i // (self.L*self.L)
        mod = i % (self.L*self.L)
        y = mod // self.L
        mod = mod % self.L
        x = mod
        return [z, y, x]

    def initialize(self, length):  # boardにオセロ の初期配置をする関数
        self.turn = WHITE
        self.ENEMY = 3 - self.turn
        self.ALLY = self.turn
        self.winner = EMPTY
        init_stone1 = self.L // 2 - 1  # 盤の中央に2*2*2で初期の石を配置する
        init_stone2 = self.L // 2  # 盤の中央に2*2*2で初期の石を配置する
        for a in range(self.y):
            for b in range(self.x):
                self.board[b][a][0] = WALL  # 盤の底面を壁に置き換え
                self.board[b][a][self.z - 1] = WALL  # 上面を
                self.board[b][0][a] = WALL  # 前面を
                self.board[b][self.y - 1][a] = WALL  # 背面を
                self.board[0][b][a] = WALL  # 左側面を
                self.board[self.x - 1][b][a] = WALL  # 右側面を

        self.board[init_stone1][init_stone2][init_stone2] = WHITE  # 中心点から左後上はシロ
        self.board[init_stone2][init_stone1][init_stone2] = WHITE  # 右前上
        self.board[init_stone2][init_stone2][init_stone1] = WHITE  # 右後下
        self.board[init_stone1][init_stone1][init_stone1] = WHITE  # 左前下
        self.board[init_stone1][init_stone1][init_stone2] = BLACK  # 中心点から左前上はクロ
        self.board[init_stone2][init_stone2][init_stone2] = BLACK  # 右後上
        self.board[init_stone1][init_stone2][init_stone1] = BLACK  # 左後下
        self.board[init_stone2][init_stone1][init_stone1] = BLACK  # 右前下

        self.direction = [[0] * ThreeD for i in range(3 ** ThreeD)]
        for i in range(len(self.direction)):
            self.direction[i] = [i // (ThreeD ** 2) - 1, (i // ThreeD) % ThreeD - 1, i % ThreeD - 1]

#           test cord for to show initial place
#        for c in range(self.z):
#            for d in range(self.y):
#                for e in range(self.x):
#                    print(symbols[self.board[e][d][c]],end=' ')
#                print("")
#            print("")

    def open(self):#ボードをターミナル上に表示する
        num_Stones = [0] * len(players)

        for i in range(self.z - 2):
            for j in range(self.y - 1):
                if j == 0:
                    print(i + 1, end=' ')
                    for k in range(self.x - 2):
                        print(k + 1, end=' ')
                else:
                    print(j, end=' ')
                    for k in range(self.x - 2):
                        if self.board[k + 1][j][i + 1] == EMPTY:
                            print(symbols[EMPTY], end=' ')
                            num_Stones[EMPTY] += 1
                        elif self.board[k + 1][j][i + 1] == WHITE:
                            print(symbols[WHITE], end=' ')
                            num_Stones[WHITE] += 1
                        elif self.board[k + 1][j][i + 1] == BLACK:
                            print(symbols[BLACK], end=' ')
                            num_Stones[BLACK] += 1
                print("")

    def chk_win(self,isDpass):#勝利判定
        num_Stones = [0] * len(players)

        for i in range(self.z - 2):
            for j in range(self.y - 2):
                for k in range(self.x - 2):
                    if self.board[k + 1][j + 1][i + 1] == EMPTY:
                        num_Stones[EMPTY] += 1
                    elif self.board[k + 1][j + 1][i + 1] == WHITE:
                        num_Stones[WHITE] += 1
                    elif self.board[k + 1][j + 1][i + 1] == BLACK:
                        num_Stones[BLACK] += 1

        if num_Stones[EMPTY] == 0 or num_Stones[WHITE] == 0 or num_Stones[BLACK] == 0 or isDpass:
            # if game is over,select winner
            self.set_winner(num_Stones)

    # def get_winner(self):


    def set_winner(self, num_Stones):#勝利判定できる盤面になればこの関数で勝者を調べる
    # test cord for to show final place
#        for c in range(self.z):
#            for d in range(self.y):
#                for e in range(self.x):
#                    print(symbols[self.board[e][d][c]],end=' ')
#                print("")
#            print("")
#
        if num_Stones[WHITE] < num_Stones[BLACK]:
            self.winner = BLACK
        elif num_Stones[WHITE] > num_Stones[BLACK]:
            self.winner = WHITE
        elif num_Stones[WHITE] == num_Stones[BLACK]:
            self.winner = WALL
        print("%d　対　%d　勝者は...%s!!!\n" %( num_Stones[WHITE],num_Stones[BLACK],players[self.winner]))
        return self.winner

    def get_possible_pos(self):
        pos=[]
        # for i in range(64):
        #     if self.board[i]==EMPTY:
        #         pos.append(i)
        for z in range(self.z-2):
            for y in range(self.y-2):
                for x in range(self.x-2):
                    point = [x+1,y+1,z+1]
                    if self.board[z+1][y+1][x+1]==EMPTY:
                        pos.append(point)
        return pos


    def chk_Cell_Ahead(self, pos, dirc):#位置pos+周囲一マスのうちのどこか(dirc)のマスを調べる
        global X, Y, Z
        return self.board[pos[X] + dirc[X]][pos[Y] + dirc[Y]][pos[Z] + dirc[Z]]

    def change_turn(self):
        # change turn counter and show whose turn is now
        self.turn = 3 - self.turn
        self.ENEMY = 3 - self.turn
        self.ALLY = self.turn
#        print("！！%sのターン！！\n" % players[self.turn])

    def set_Next_Position(self, pos, dirc):
        pos = [pos[X] + dirc[X],pos[Y] + dirc[Y],pos[Z] + dirc[Z]]
        return pos
    
    def greedy(choice):
        num_stone_get = []
        greedychoices = []
        for i in choice:
            num_stone_get.append(len(get_flip_list(choice)))
        for j in range(len(choice)):
            if num_stone_get[j] == max(num_stone_get):
                greedychoices.append(choice[j])
        return greedychoices
    def humble(choice):
        num_stone_get = []
        greedychoices = []
        for i in choice:
            num_stone_get.append(len(get_flip_list(choice)))
        for j in range(len(choice)):
            if num_stone_get[j] == min(num_stone_get):
                greedychoices.append(choice[j])
        return greedychoices
            
            
    
    def can_put_stone_all(self):
        points = []
        for z in range(self.z-2):
            for y in range(self.y-2):
                for x in range(self.x-2):
                    point = [x+1,y+1,z+1]
                    if self.can_put_stone(point):
                        points.append(point)
        return points


    def can_put_stone(self, pos):
        here = [0, 0, 0]
        result = False

        if self.chk_Cell_Ahead(pos, here) == EMPTY:
            for i in range(len(self.direction)):
                if self.direction[i] != here:
                    p = self.set_Next_Position(pos, here)
                    while (self.chk_Cell_Ahead(p, self.direction[i]) == self.ENEMY):
                        p = self.set_Next_Position(p, self.direction[i])
                        if self.chk_Cell_Ahead(p, self.direction[i]) == self.ALLY:
                            result = True
                if result == True:
                    break

        return result

    def flip(self, pos):
        here = [0, 0, 0]
        self.setCell(pos, self.ALLY)
        
        flip_pos_list = [pos]
        for i in range(len(self.direction)):
            if self.direction[i] != here:
                p = self.set_Next_Position(pos, here)
                pos_list = []
                while (self.chk_Cell_Ahead(p, self.direction[i]) == self.ENEMY):
                    p = self.set_Next_Position(p, self.direction[i])
                    pos_list.append(p)
                    if self.chk_Cell_Ahead(p, self.direction[i]) == self.ALLY:
                        for j in pos_list:
                            flip_pos_list.append(j)
        for k in flip_pos_list:
            self.setCell(k, self.ALLY)

    def get_flip_list(self, pos):
        here = [0, 0, 0]
        self.setCell(pos, self.ALLY)
        
        flip_pos_list = [pos]
        for i in range(len(self.direction)):
            if self.direction[i] != here:
                p = self.set_Next_Position(pos, here)
                pos_list = []
                while (self.chk_Cell_Ahead(p, self.direction[i]) == self.ENEMY):
                    p = self.set_Next_Position(p, self.direction[i])
                    pos_list.append(p)
                    if self.chk_Cell_Ahead(p, self.direction[i]) == self.ALLY:
                        for j in pos_list:
                            flip_pos_list.append(j)
                            
        return flip_pos_list

class In_Out_put:

    def __init__(self,val):
        self.QUIT = -1
        self.PASS = -2
        self.COMMAND = 2
        self.ONEMORE = 3
        self.PCQUIT = False
        if val == 0:
            self.LENGTH = self.select_size()
        else:
            self.LENGTH = (val+2)*2

    def is_position(self, char):
        result = False
        try:
            chk_char = int(char)
        except ValueError as e:
            return result
        for i in range(self.LENGTH - 2):
            if chk_char is i + 1:
                result = True
        return result

    def read_command(self, val):
        quit_or_pass_or_command = -1  # error flag
        if val == "quit":
            quit_or_pass_or_command = self.QUIT
        elif val == "pass":
            quit_or_pass_or_command = self.PASS
        elif self.is_position(val):
            quit_or_pass_or_command = self.COMMAND
        else:
            quit_or_pass_or_command = self.ONEMORE
        return quit_or_pass_or_command

    def read(self):
        i = 0
        coordinate_input = [0] * ThreeD
        read_command_up = True
        while read_command_up:
            if i == X:
                print("x座標を入力してください")
            elif i == Y:
                print("y座標を入力してください")
            elif i == Z:
                print("z座標を入力してください")
            val = input()

            if self.read_command(val) == self.QUIT:
                self.PCQUIT = True
                break
            if self.read_command(val) == self.PASS:
                coordinate_input = [self.PASS] * ThreeD
                break
            if self.read_command(val) == self.COMMAND:
                coordinate_input[i] = int(val)
                i = i + 1
            if self.read_command(val) == self.ONEMORE:
                print("指定した数または文字は不正です。")
            if i == 3:
                read_command_up = False
        return coordinate_input

    def select_size(self):
        size = 0

        print("ゲーム盤のサイズを左記の数字より指定してください　小{1,2,3}大")
        val = input()
        while True:
            value_is_int = True
            try:
                chk_char = int(val)
            except ValueError as e:
                value_is_int = False
            if value_is_int:
                if int(val) == 3 or int(val) == 2 or int(val) == 1:
                    break
            print("不正な値です")
            val = input()

        size = (int(val) + 2) * 2
        return size

import random
class RandomPlayer:
    def __init__(self, turn):
        self.name = "Random"
        self.myturn = turn
        self.PASS = -2

    def act(self, board):
        acts = board.can_put_stone_all()
        if len(acts) == 0:
            acts.append([self.PASS,self.PASS,self.PASS])
        i = random.randrange(len(acts))
        return acts[i]

    def getGameResult(self, board):
        pass
    
import numpy as np


class GreedyPlayer:
    def __init__(self,turn):
        self.name = "Greedy"
        self.myturn = turn
        self.PASS = -2
    def act(self,board):
        choice = board.can_put_stone_all()
        if len(acts) == 0:
            acts.append([self.PASS,self.PASS,self.PASS])
        else:
            acts = board.greedy(choice)
            i = random.randrange(len(acts))
        return acts[i]
    def getGameResult(self, board):
        pass
class HumblePlayer:
    def __init__(self,turn):
        self.name = "Greedy"
        self.myturn = turn
        self.PASS = -2
    def act(self,board):
        choice = board.can_put_stone_all()
        if len(acts) == 0:
            acts.append([self.PASS,self.PASS,self.PASS])
        else:
            acts = board.greedy(choice)
            i = random.randrange(len(acts))
        return acts[i]
    def getGameResult(self, board):
        pass
    
    
    

class Players_Osero_game:
    def __init__(self,size_val):
        self.PASS = -2
        self.passmater = 0
        self.isDpass = False
        self.reader = In_Out_put(size_val)
        self.L = self.reader.LENGTH
        self.board = Board(self.L)
#        self.print_Usage()
        self.players = [DQNPlayer(WHITE), RandomPlayer(BLACK)]

    # L = (L-2)^3 width board will crate

    def is_continue(self):
        self.board.chk_win(self.isDpass)
        if self.board.winner == EMPTY:
            return True
        else:
            return False

    def is_not_Quit(self):
        ans = not self.reader.PCQUIT
        return ans

    def update(self):
        
        if self.board.turn == WHITE:
            input_pos = self.players[0].act(self.board)
        elif self.board.turn == BLACK:
            input_pos = self.players[1].act(self.board)


        if type(input_pos) is np.int64:
            input_pos = self.board.get_unflatten_point(input_pos)
        if self.is_not_Quit() == False:
            print("Thank you for playing")
            self.passmater = 0
        elif input_pos == [self.PASS, self.PASS, self.PASS]:
            self.board.change_turn()
            self.passmater += 1
            if self.passmater == 2:
                self.isDpass = True
            
        elif self.board.can_put_stone(input_pos):
            self.board.flip(input_pos)
            self.board.change_turn()
            self.passmater = 0
        else:
            self.passmater = 0
            print("指定した座標には置けませんでした")
#        self.board.open() #ここで盤面を表示

    def print_Usage(self):
        print("---    遊び方    ---\n パス: pass \n ゲームの終了 : quit\n 座標を入れる : 1 ~ %d\n" % (self.L - 1))


In [2]:
import chainer

from chainer import Function, gradient_check, Variable, optimizers, serializers, utils
import chainer.functions as F  # Functionは、パラメータを持たない関数です。
import chainer.links as L  # links パラメーターを持つ関数
import numpy as np
from chainer import computational_graph as c

# Network definition, Multilayer perceptron
class MLP(chainer.Chain):
    # L.linear(input_dim_num, out_dim_num) 全結合層
    def __init__(self, n_in, n_units, n_out):
        super(MLP, self).__init__(
            l1=L.Linear(n_in, n_units),  # first layer
            l2=L.Linear(n_units, n_units),  # second layer
            l3=L.Linear(n_units, n_units),  # Third layer
            l4=L.Linear(n_units, n_out),  # output layer
        )
    """
    mean squad error = 二乗誤差
    leaky relu : reluの一つ
    L -> F -> L
    """
    def __call__(self, x, t=None, train=False):
        h = F.leaky_relu(self.l1(x))
        h = F.leaky_relu(self.l2(h))
        h = F.leaky_relu(self.l3(h))
        h = self.l4(h)

        if train:
            return F.mean_squared_error(h,t)
        else:
            return h

    def get(self,x):
        # input x as float, output float
        return self.predict(Variable(np.array([x]).astype(np.float32).reshape(1,1))).data[0][0]


class DQNPlayer:
    def __init__(self, turn,name="DQN",e=1,dispPred=False):
        self.name=name
        self.myturn=turn
        self.model = MLP(64, 256, 64)
        self.optimizer = optimizers.SGD()
        self.optimizer.setup(self.model)
        self.e=e
        self.gamma=0.95
        self.dispPred=dispPred
        self.last_move=None
        self.last_board=None
        self.last_pred=None
        self.totalgamecount=0
        self.rwin,self.rlose,self.rdraw,self.rmiss=1,-1,0,-1.5
        
    
    def act(self,board):
        
        self.last_board = board.clone()
        x=np.array([board.board],dtype=np.float32).astype(np.float32)
        
        pred=self.model(x)
        if self.dispPred:print(pred.data)
        self.last_pred=pred.data[0,:]
        act=np.argmax(pred.data,axis=1)
        if self.e > 0.2: #decrement epsilon over time
            self.e -= 1/(20000)
        if random.random() < self.e:
            acts=board.get_possible_pos()
            i=random.randrange(len(acts))
            act=acts[i]
        i=0
        while board.board[act]!=EMPTY:
            #print("Wrong Act "+str(board.board)+" with "+str(act))
            self.learn(self.last_board,act, -1, self.last_board)
            x=np.array([board.board],dtype=np.float32).astype(np.float32)
            pred=self.model(x)  # modelに対してデータのみを渡すと予測を行う。ここだと、盤面データから次の手の予測を行う。
            #print(pred.data)
            act=np.argmax(pred.data,axis=1)
            i+=1
            if i>10:
#                print("Exceed Pos Find"+str(board.board)+" with "+str(act))
                acts=self.last_board.get_possible_pos()
                act=acts[random.randrange(len(acts))]
            
        self.last_move=act
        #self.last_pred=pred.data[0,:]
        return act
    
    def getGameResult(self,board):
        r=0
        if self.last_move is not None:
            if board.winner is None:
                self.learn(self.last_board,self.last_move, 0, board)
                pass
            else:
                if board.board== self.last_board.board:            
                    self.learn(self.last_board,self.last_move, self.rmiss, board)
                elif board.winner == self.myturn:
                    self.learn(self.last_board,self.last_move, self.rwin, board)
                elif board.winner !=DRAW:
                    self.learn(self.last_board,self.last_move, self.rlose, board)
                else:#DRAW
                    self.learn(self.last_board,self.last_move, self.rdraw, board)
                self.totalgamecount+=1
                self.last_move=None
                self.last_board=None
                self.last_pred=None

    def learn(self,s,a,r,fs):
        if fs.winner is not None:
            maxQnew=0
        else:
            x=np.array([fs.board],dtype=np.float32).astype(np.float32)
            maxQnew=np.max(self.model(x).data[0])
        update=r+self.gamma*maxQnew
        #print(('Prev Board:{} ,ACT:{}, Next Board:{}, Get Reward {}, Update {}').format(s.board,a,fs.board,r,update))
        #print(('PREV:{}').format(self.last_pred))
        self.last_pred[a]=update
        
        x=np.array([s.board],dtype=np.float32).astype(np.float32)   # データ
        t=np.array([self.last_pred],dtype=np.float32).astype(np.float32)   # 教師
        self.model.zerograds()
        loss=self.model(x,t,train=True)
        loss.backward()
        self.optimizer.update()
        

/Users/yusuke/.pyenv/versions/3.8.1/lib/python3.8/site-packages/chainer/_environment_check.py:33: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BLAS libraries such as OpenBLAS.
For details of the issue, please see
https://docs.chainer.org/en/stable/tips.html#mnist-example-does-not-converge-in-cpu-mode-on-mac-os-x.

Please be aware that Mac OS X is not an officially supported OS.

  warnings.warn('''\


In [3]:
if __name__ == '__main__':
    size = 1 # 1:4*4*4  2:6*6*6  3:8*8*8
    result = []
    for _ in range(100):
        game_a = Players_Osero_game(size)
        while (game_a.is_continue() and game_a.is_not_Quit()):
            game_a.update()
        result.append(game_a.board.winner)

    print("Whiteの勝利数: {}".format(result.count(1)))
    print("Blackの勝利数: {}".format(result.count(2)))



AttributeError: 'Board' object has no attribute 'clone'

3

# for i in range(10):
    fuga = False
    if i == 3 and fuga == False:
        print(hoge)
        i = i-3
    print(i)

In [ ]:
hoge = [1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,0,0,1,1,0,1,0,1]
line_b = []
for i in hoge:
    line_b.append(i)
line_b

In [ ]:
import copy
hoge = [1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,0,0,1,1,0,1,0,1]

len(hoge)
def centerize(line):
    line_b = copy.copy(line)
    for i in range(len(line_b)):
        if line_b[i] < line_b[i-1]:
            line[i] = line_b[i-1]
            line[i-1] = line_b[i]
    return line
for i in range(100):
    hoge = centerize(hoge)
strhoge = ""
for i in hoge:
    strhoge += str(i)
strhoge
print(int(strhoge, 2))
print(hoge)